In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/selectedmitbih/resized_data.h5
/kaggle/input/selectedmitbih/test_dataset.h5


In [2]:
import h5py
import numpy as np

In [3]:
with h5py.File('/kaggle/input/selectedmitbih/resized_data.h5', 'r') as file:
    train_data = file['resized_dataset']
    train_labels = file['beat_type']
    train_data = np.array(train_data)
    train_labels = np.array(train_labels)


In [4]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
import numpy as np
from tensorflow.keras.optimizers import Adam
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


2024-03-29 06:01:40.796281: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-29 06:01:40.796384: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-29 06:01:40.939119: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
ros = RandomOverSampler()
train_data, train_labels = ros.fit_resample(train_data.reshape(-1, 224*224*3), train_labels)
train_data = train_data.reshape(-1, 224, 224, 3)

# Split the data into train and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

# Define the number of classes
num_classes = 3
train_labels = to_categorical(train_labels, num_classes=num_classes)
val_labels =  to_categorical(val_labels, num_classes=num_classes)

# Load pre-trained ResNet50 model with weights from ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = True

# Add new fully-connected layers
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)  # Global average pooling to reduce dimensions
x = Dense(1024, activation='relu')(x)  # Add new fully-connected layer
x = Dense(512, activation='relu')(x)  # Add another fully-connected layer
predictions = Dense(num_classes, activation='softmax')(x)  # Add softmax layer for classification

model = Model(inputs=base_model.input, outputs=predictions)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
with h5py.File('/kaggle/input/selectedmitbih/test_dataset.h5', 'r') as file:
    test_data = file['resized_dataset']
    test_labels = file['beat_type']
    test_data = np.array(test_data)
    test_labels = np.array(test_labels)
    
test_labels = to_categorical(test_labels, num_classes=3)


In [7]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam, SGD
from sklearn.metrics import classification_report
import os

# Define a list of learning rates and optimizers to try
learning_rates = [0.001, 0.01, 0.1]
optimizers = [Adam, SGD]

best_model_path = 'best_model.keras'
best_accuracy = 0
best_hyperparameters = {}

for lr in learning_rates:
    for optimizer_class in optimizers:
        # Instantiate the optimizer with the current learning rate
        optimizer = optimizer_class(learning_rate=lr)
        
        # Compile the model with the optimizer and loss function
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        # Add callbacks for learning rate reduction, early stopping, and model checkpoint
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)
        early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
        checkpoint = ModelCheckpoint(best_model_path, monitor='val_loss', save_best_only=True, mode='min')
        
        # Fit the data to the model
        history = model.fit(train_data, train_labels, batch_size=32, epochs=2, 
                            validation_data=(val_data, val_labels), 
                            callbacks=[reduce_lr, early_stop, checkpoint])
        
        # Evaluate the model on validation data
        val_loss, _ = model.evaluate(val_data, val_labels)
        
        predictions = model.predict(test_data)
        predicted_labels = tf.argmax(predictions, axis=1)
        true_labels = tf.argmax(test_labels, axis=1)  # Assuming you have true labels for comparison
        print('prediction complete')

        # Calculate evaluation metrics
        report = classification_report(true_labels, predicted_labels, output_dict=True)

        f1_score = report['weighted avg']['f1-score']
        precision = report['weighted avg']['precision']
        positive_predictive_value = report['weighted avg']['precision']
        specificity = report['weighted avg']['recall']
        sensitivity = report['weighted avg']['recall']
        accuracy = report['accuracy']
        
        
        # Check if current model has the lowest validation loss
        if best_accuracy < accuracy:
            # Update best validation loss and hyperparameters
            best_accuracy = accuracy
            best_hyperparameters = {'learning_rate': lr, 'optimizer': optimizer_class.__name__}
            # Remove previous best model if exists
            if os.path.exists(best_model_path):
                os.remove(best_model_path)
                

Epoch 1/2


I0000 00:00:1711692259.692923      69 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


416/416 ━━━━━━━━━━━━━━━━━━━━ 175s 258ms/step - accuracy: 0.9441 - loss: 0.2182 - val_accuracy: 0.9910 - val_loss: 0.0449 - learning_rate: 0.0010
Epoch 2/2
416/416 ━━━━━━━━━━━━━━━━━━━━ 65s 157ms/step - accuracy: 0.9923 - loss: 0.0276 - val_accuracy: 0.7673 - val_loss: 1.9042 - learning_rate: 0.0010
Restoring model weights from the end of the best epoch: 1.
104/104 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.9899 - loss: 0.0404
1104/1104 ━━━━━━━━━━━━━━━━━━━━ 48s 40ms/step
prediction complete
Epoch 1/2
416/416 ━━━━━━━━━━━━━━━━━━━━ 107s 198ms/step - accuracy: 0.9933 - loss: 0.0310 - val_accuracy: 0.9916 - val_loss: 0.0321 - learning_rate: 0.0010
Epoch 2/2
416/416 ━━━━━━━━━━━━━━━━━━━━ 63s 151ms/step - accuracy: 0.9946 - loss: 0.0228 - val_accuracy: 0.9919 - val_loss: 0.0284 - learning_rate: 0.0010
Restoring model weights from the end of the best epoch: 2.
104/104 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.9918 - loss: 0.0276
1104/1104 ━━━━━━━━━━━━━━━━━━━━ 48s 41ms/step
prediction c

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/2
416/416 ━━━━━━━━━━━━━━━━━━━━ 107s 195ms/step - accuracy: 0.3192 - loss: 1.1006 - val_accuracy: 0.3307 - val_loss: 1.0991 - learning_rate: 0.1000
Epoch 2/2
416/416 ━━━━━━━━━━━━━━━━━━━━ 63s 151ms/step - accuracy: 0.3301 - loss: 1.0992 - val_accuracy: 0.3443 - val_loss: 1.0987 - learning_rate: 0.1000
Restoring model weights from the end of the best epoch: 2.
104/104 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.3488 - loss: 1.0982
1104/1104 ━━━━━━━━━━━━━━━━━━━━ 48s 40ms/step
prediction complete


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
print(best_hyperparameters)
print(best_accuracy)

{'learning_rate': 0.01, 'optimizer': 'SGD'}
0.803799977347378


In [9]:
# import tensorflow as tf
# from sklearn.metrics import classification_report

# best_model = load_model(best_model_path)

# with h5py.File('/kaggle/input/selectedmitbih/test_dataset.h5', 'r') as file:
#     test_data = file['resized_dataset']
#     test_labels = file['beat_type']
#     test_data = np.array(test_data)
#     test_labels = np.array(test_labels)
    
# test_labels = to_categorical(test_labels, num_classes=3)

# print('start prediction')
# # Evaluate the model on the test data
# predictions = best_model.predict(test_data)
# predicted_labels = tf.argmax(predictions, axis=1)
# true_labels = tf.argmax(test_labels, axis=1)  # Assuming you have true labels for comparison
# print('prediction complete')

# # Calculate evaluation metrics
# report = classification_report(true_labels, predicted_labels, output_dict=True)

# f1_score = report['weighted avg']['f1-score']
# precision = report['weighted avg']['precision']
# positive_predictive_value = report['weighted avg']['precision']
# specificity = report['weighted avg']['recall']
# sensitivity = report['weighted avg']['recall']
# accuracy = report['accuracy']

# print(f"F1 Score: {f1_score}")
# print(f"Precision: {precision}")
# print(f"Positive Predictive Value: {positive_predictive_value}")
# print(f"Specificity: {specificity}")
# print(f"Sensitivity: {sensitivity}")
# print(f"Accuracy: {accuracy}")

In [10]:
# # Define optimizer with specified learning rate
# learning_rate = 0.001
# optimizer = Adam(learning_rate=learning_rate)

# # Compile the model with the optimizer and loss function
# model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# # Add callbacks for learning rate reduction and early stopping based on validation loss
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)
# early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# # Fit the data to the model
# history = model.fit(train_data, train_labels, batch_size=32, epochs=40, validation_data=(val_data, val_labels), callbacks=[reduce_lr, early_stop])

In [11]:
# # Access the best model
# model = grid_result.best_estimator_.model

# # Save the best model
# model.save('best_model.h5')

In [12]:
# !zip -r file.zip /kaggle/working

In [13]:
# !ls

In [14]:
# from IPython.display import FileLink
# FileLink(r'file.zip')

In [15]:
# import tensorflow as tf
# from sklearn.metrics import classification_report
# print('Read dataset complete')

# with h5py.File('/kaggle/input/selectedmitbih/test_dataset.h5', 'r') as file:
#     test_data = file['resized_dataset']
#     test_labels = file['beat_type']
#     test_data = np.array(test_data)
#     test_labels = np.array(test_labels)
    
# test_labels = to_categorical(test_labels, num_classes=3)

# print('start prediction')
# # Evaluate the model on the test data
# predictions = model.predict(test_data)
# predicted_labels = tf.argmax(predictions, axis=1)
# true_labels = tf.argmax(test_labels, axis=1)  # Assuming you have true labels for comparison
# print('prediction complete')

# # Calculate evaluation metrics
# report = classification_report(true_labels, predicted_labels, output_dict=True)

# f1_score = report['weighted avg']['f1-score']
# precision = report['weighted avg']['precision']
# positive_predictive_value = report['weighted avg']['precision']
# specificity = report['weighted avg']['recall']
# sensitivity = report['weighted avg']['recall']
# accuracy = report['accuracy']

# print(f"F1 Score: {f1_score}")
# print(f"Precision: {precision}")
# print(f"Positive Predictive Value: {positive_predictive_value}")
# print(f"Specificity: {specificity}")
# print(f"Sensitivity: {sensitivity}")
# print(f"Accuracy: {accuracy}")